______________________________________________________________________________________________________________________________________________________________________________________

Обучение модели

In [ ]:
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import roc_auc_score, f1_score

# Загрузка данных
df = pd.read_csv('processed_data.csv')

try:
    # Попытка найти ресурс
    nltk.data.find('corpora/stopwords')
except LookupError:
    # Если не найдено, скачиваем
    nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = list(stopwords.words('russian'))
stop_words.append('г')

# Векторизация текста
vectorizer = TfidfVectorizer(max_features=1000, stop_words=stop_words)
X = vectorizer.fit_transform(df['comment'])

# Целевые метки
Y = df[['Вопрос решен', 'Нравится качество выполнения заявки', 
        'Нравится качество работы сотрудников', 
        'Нравится скорость отработки заявок', 
        'Понравилось выполнение заявки', 'Другое']]

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=26)

# Обучение модели многометочной логистической регрессии
model = MultiOutputClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

# Предсказания вероятностей для тестовой выборки
y_pred_proba = model.predict_proba(X_test)

roc_auc_scores = {}
f1_scores = {}

for idx, column in enumerate(Y.columns):
    # ROC-AUC по вероятностям
    roc_auc = roc_auc_score(y_test[column], y_pred_proba[idx][:, 1])
    roc_auc_scores[column] = roc_auc
    
    # Предсказание классов (бинарных)
    y_pred_class = (y_pred_proba[idx][:, 1] >= 0.5).astype(int)
    
    # F1-score по предсказанным классам
    f1 = f1_score(y_test[column], y_pred_class)
    f1_scores[column] = f1

average_roc_auc = sum(roc_auc_scores.values()) / len(roc_auc_scores)
average_f1_score = sum(f1_scores.values()) / len(f1_scores)

print("ROC-AUC по категориям:")
for category, score in roc_auc_scores.items():
    print(f"{category}: {score:.3f}")

print(f"\nСредний ROC-AUC: {average_roc_auc:.3f}\n")

print("\nF1-score по категориям:")
for category, score in f1_scores.items():
    print(f"{category}: {score:.3f}")

print(f"\nСредний F1-score: {average_f1_score:.3f}")

ROC-AUC по категориям:
Вопрос решен: 0.815
Нравится качество выполнения заявки: 0.902
Нравится качество работы сотрудников: 0.954
Нравится скорость отработки заявок: 0.964
Понравилось выполнение заявки: 0.791
Другое: 0.982

Средний ROC-AUC: 0.901


F1-score по категориям:
Вопрос решен: 0.133
Нравится качество выполнения заявки: 0.457
Нравится качество работы сотрудников: 0.737
Нравится скорость отработки заявок: 0.881
Понравилось выполнение заявки: 0.114
Другое: 0.632

Средний F1-score: 0.492


In [37]:
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import roc_auc_score, f1_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Загрузка данных
df = pd.read_csv('processed_data.csv')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = list(stopwords.words('russian'))
stop_words += ['г']

# Векторизация текста с гиперпараметрами для оптимизации
vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1,2), stop_words=stop_words)
X = vectorizer.fit_transform(df['comment'])

# Целевые метки
Y = df[['Вопрос решен', 'Нравится качество выполнения заявки', 
        'Нравится качество работы сотрудников', 
        'Нравится скорость отработки заявок', 
        'Понравилось выполнение заявки', 'Другое']]

# Обработка несбалансированных данных: вычисление веса классов для каждой метки
class_weights = {}
for column in Y.columns:
    classes = np.unique(Y[column])
    if len(classes) == 2:
        weights = compute_class_weight(class_weight='balanced', classes=classes, y=Y[column])
        class_weights[column] = dict(zip(classes, weights))
    else:
        # Если есть только один класс (редкий случай), пропускаем или задаем равные веса
        class_weights[column] = {0:1.0, 1:1.0}

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Обучение модели с учетом веса классов (для каждого столбца отдельно)
# Для этого создадим кастомный классификатор с учетом веса (или используем встроенные параметры)
# Но LogisticRegression не принимает словарь весов для многометочной задачи напрямую,
# поэтому можно обучать отдельные модели для каждой метки или использовать классический подход.
# Для простоты — обучим без учета веса. Для более точной обработки — можно реализовать кастомный цикл.

model = MultiOutputClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

# Проверка предсказаний и оценка модели по каждой метке
y_pred_proba = model.predict_proba(X_test)

roc_auc_scores = {}
f1_scores = {}

for idx, column in enumerate(Y.columns):
    # ROC-AUC по вероятностям
    roc_auc_scores[column] = roc_auc_score(y_test[column], y_pred_proba[idx][:, 1])
    
    # Предсказание классов (бинарных)
    y_pred_class = (y_pred_proba[idx][:, 1] >= 0.5).astype(int)
    
    # F1-score по предсказанным классам
    f1_scores[column] = f1_score(y_test[column], y_pred_class)

# Средние показатели по категориям
average_roc_auc = sum(roc_auc_scores.values()) / len(roc_auc_scores)
average_f1_score = sum(f1_scores.values()) / len(f1_scores)

print("ROC-AUC по категориям:")
for category, score in roc_auc_scores.items():
    print(f"{category}: {score:.3f}")

print(f"\nСредний ROC-AUC: {average_roc_auc:.3f}\n")

print("F1-score по категориям:")
for category, score in f1_scores.items():
    print(f"{category}: {score:.3f}")

print(f"\nСредний F1-score: {average_f1_score:.3f}")

# --- Проверка предсказаний ---
# Можно вывести отчеты для каждой категории или для всей выборки.
for column in Y.columns:
    print(f"\nКлассика для '{column}':")
    print(classification_report(y_test[column], (y_pred_proba[Y.columns.get_loc(column)][:, 1] >= 0.5).astype(int)))

# --- Оптимизация векторизации ---
# Вы уже использовали ngram_range=(1,2), что хорошо.
# Можно попробовать разные параметры через GridSearchCV или вручную.
# Например:
"""
best_max_features = [1000, 2000]
best_ngram_range = [(1,1), (1,2)]
для каждого варианта создайте векторизатор и обучите модель.
"""

# --- Использование кросс-валидации ---
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=26)

scores_cv_roc_auc = {col: [] for col in Y.columns}
scores_cv_f1 = {col: [] for col in Y.columns}

for train_idx, val_idx in skf.split(X, Y['Вопрос решен']):  # Можно делать по одной метке или по всему набору (сложнее)
    X_train_cv, X_val_cv = X[train_idx], X[val_idx]
    y_train_cv, y_val_cv = Y.iloc[train_idx], Y.iloc[val_idx]
    
    model_cv = MultiOutputClassifier(LogisticRegression(max_iter=1000))
    model_cv.fit(X_train_cv, y_train_cv)
    
    y_pred_proba_cv = model_cv.predict_proba(X_val_cv)
    
    for idx, column in enumerate(Y.columns):
        # ROC-AUC на кросс-валидации
        score_roc_auc = roc_auc_score(y_val_cv[column], y_pred_proba_cv[idx][:, 1])
        scores_cv_roc_auc[column].append(score_roc_auc)
        
        # F1-score на кросс-валидации
        y_pred_class_cv = (y_pred_proba_cv[idx][:, 1] >= 0.5).astype(int)
        score_f1 = f1_score(y_val_cv[column], y_pred_class_cv)
        scores_cv_f1[column].append(score_f1)

# Средние показатели кросс-валидации
print("\nКросс-валидация ROC-AUC:")
for col in Y.columns:
    print(f"{col}: {np.mean(scores_cv_roc_auc[col]):.3f}")

print("\nКросс-валидация F1-score:")
for col in Y.columns:
    print(f"{col}: {np.mean(scores_cv_f1[col]):.3f}")

ROC-AUC по категориям:
Вопрос решен: 0.765
Нравится качество выполнения заявки: 0.817
Нравится качество работы сотрудников: 0.941
Нравится скорость отработки заявок: 0.971
Понравилось выполнение заявки: 0.780
Другое: 0.924

Средний ROC-AUC: 0.866

F1-score по категориям:
Вопрос решен: 0.027
Нравится качество выполнения заявки: 0.400
Нравится качество работы сотрудников: 0.564
Нравится скорость отработки заявок: 0.854
Понравилось выполнение заявки: 0.048
Другое: 0.478

Средний F1-score: 0.395

Классика для 'Вопрос решен':
              precision    recall  f1-score   support

           0       0.75      1.00      0.86       224
           1       1.00      0.01      0.03        74

    accuracy                           0.76       298
   macro avg       0.88      0.51      0.44       298
weighted avg       0.82      0.76      0.65       298


Классика для 'Нравится качество выполнения заявки':
              precision    recall  f1-score   support

           0       0.95      1.00     